# `Enum.map`と再帰スタイルの比較、ふたたび

「[Elixirで速度を追い求めるときのプログラミングスタイル - Qiita](https://qiita.com/zacky1972/items/5963a8bf5f2a34c67d88#enummap%E3%81%A8%E5%86%8D%E5%B8%B0%E3%82%B9%E3%82%BF%E3%82%A4%E3%83%AB%E3%81%AF%E3%81%BB%E3%81%A8%E3%82%93%E3%81%A9%E5%B7%AE%E3%81%8C%E3%81%AA%E3%81%84)」という記事にある、「`Enum.map`と再帰スタイルは，ほとんど差がない」という検証結果が興味深かったので、自分でもやってみました。興味深かったポイントは、末尾再帰にしたらやっぱり速くなるのではないか？ということです。

## ベンチマーク対象のコード

### `Enum.map`による実装

```elixir
Enum.map(@list, & &1 * 2)
```

### 再帰による実装

[lib/recursive.ex](lib/recursive.ex)

```elixir
defmodule Recursive do
  def r_map([], _func), do: []

  def r_map([head | tail], func) do
    [func.(head) | r_map(tail, func)]
  end
end

Recursive.r_map(@list, & &1 * 2)
```

ここまでは元記事の通りです。

### 末尾再帰による実装

[lib/tail_recursive.ex](lib/tail_recursive.ex)

```elixir
defmodule TailRecursive do
  def r_map(list, func) do
    r_map_rec(list, func, [])
    |> Enum.reverse()
  end

  defp r_map_rec([], _func, acc), do: acc

  defp r_map_rec([head | tail], func, acc) do
    r_map_rec(tail, func, [func.(head) | acc])
  end
end

TailRecursive.r_map(@list, & &1 * 2)
```

こちらが、あらたに追加した実装です。

### テストによる実装の確認

そもそも実装がちゃんとできているのか、テストしてみます。

```elixir
defmodule RMapBenchTest do
  use ExUnit.Case

  setup_all do
    %{list: Enum.to_list(1..100)}
  end

  test "all functions return same values", %{list: list} do
    l1 = Enum.map(list, &(&1 * 2))
    l2 = Recursive.r_map(list, &(&1 * 2))
    l3 = TailRecursive.r_map(list, &(&1 * 2))
    assert l1 == l2
    assert l2 == l3
    assert l1 == l3
  end
end
```

In [1]:
!mix test

.

Finished in 0.04 seconds
1 test, 0 failures

Randomized with seed 985632


実装はOKなようですね。

## ベンチマーク

元記事同様、[benchfella](https://github.com/alco/benchfella)を使ってベンチマークします。

[bench/r_map_bench.ex](bench/r_map_bench.ex)

```elixir
defmodule EnumBench do
  use Benchfella

  @list Enum.to_list(1..1_000_000)

  bench "Enum.map" do
    Enum.map(@list, & &1 * 2)
  end

  bench "recursive" do
    Recursive.r_map(@list, & &1 * 2)
  end

  bench "tail recursive" do
    TailRecursive.r_map(@list, & &1 * 2)
  end
end
```

以下の通り、3回実行してみました（[bench/snapshots](bench/snapshots)に結果が格納されています）。

### 1回目

In [2]:
!mix bench

Settings:
  duration:      1.0 s

## EnumBench
[19:21:17] 1/3: Enum.map
[19:21:21] 2/3: recursive
[19:21:24] 3/3: tail recursive

Finished in 10.31 seconds

## EnumBench
benchmark name  iterations   average time 
tail recursive          50   48389.76 µs/op
recursive               50   56384.84 µs/op
Enum.map                50   58459.10 µs/op



### 2回目

In [3]:
!mix bench

Settings:
  duration:      1.0 s

## EnumBench
[19:21:51] 1/3: Enum.map
[19:21:55] 2/3: recursive
[19:21:58] 3/3: tail recursive

Finished in 10.32 seconds

## EnumBench
benchmark name  iterations   average time 
tail recursive          50   47577.26 µs/op
recursive               50   57106.66 µs/op
Enum.map                50   57772.08 µs/op



### 3回目

In [4]:
!mix bench

Settings:
  duration:      1.0 s

## EnumBench
[19:22:26] 1/3: Enum.map
[19:22:30] 2/3: recursive
[19:22:33] 3/3: tail recursive

Finished in 10.35 seconds

## EnumBench
benchmark name  iterations   average time 
tail recursive          50   58784.60 µs/op
Enum.map                50   70148.52 µs/op
recursive               20   71049.85 µs/op



## 結果の考察

元記事にある通り、`Enum.map`と再帰とでは、あまり変わりがないように見えます。しかし、末尾再帰による実装では、やや目に見えて速くなっているようにも思われます。このことから、パフォーマンスを追求する際には、`Enum.map`ではなく末尾再帰となる再帰による実装が有効であるといえるようにも思われました。